In [53]:
import numpy as np
import pandas as pd
import yfinance as yf
import pandas_ta as ta
import matplotlib.pyplot as plt
import tabulate as tb

In [54]:
TICKER = "AAPL"
TIMEFRAME = "240min"    # 1d, 1h, 15m, 5m
START_DATE = "2022-03-07"
END_DATE = "2025-03-07"
PATH_IN = f'./../out_data/{TICKER}_{TIMEFRAME}.csv'
PATH_OUT = f'./../data/{TICKER}_{TIMEFRAME}.csv'

In [55]:
df = pd.read_csv(PATH_IN, parse_dates=['datetime'])
df = df.rename(columns=lambda x: x.capitalize())

df = df[df['Datetime'] >= pd.to_datetime(START_DATE)]
df = df[df['Datetime'] <= pd.to_datetime(END_DATE)]

df.set_index('Datetime', inplace=True)
df = df.sort_index()

print(tb.tabulate(df.tail(), headers='keys', tablefmt='psql'))

+---------------------+--------+--------+--------+---------+------------+
| Datetime            |   Open |   High |    Low |   Close |   % change |
|---------------------+--------+--------+--------+---------+------------|
| 2025-03-04 18:00:00 | 236.61 | 239.24 | 234.71 |  235.82 | -0.333883  |
| 2025-03-05 14:00:00 | 235.13 | 235.4  | 229.24 |  234.94 | -0.0808064 |
| 2025-03-05 18:00:00 | 234.93 | 236.5  | 234.04 |  236.02 |  0.463968  |
| 2025-03-06 14:00:00 | 234.2  | 237.84 | 233.77 |  235.03 |  0.354398  |
| 2025-03-06 18:00:00 | 235.01 | 235.96 | 233.31 |  235.63 |  0.263819  |
+---------------------+--------+--------+--------+---------+------------+


In [ ]:
df["rsi_14"] = ta.rsi(df["Close"], length=14)
df["rsi_28"] = ta.rsi(df["Close"], length=28)
df["rsi_50"] = ta.rsi(df["Close"], length=50)
df["rsi_7"] = ta.rsi(df["Close"], length=7)

macd = ta.macd(df["Close"])
# df = df.join(macd)
df["macd"] = macd["MACD_12_26_9"]

df["ema_10"] = ta.ema(df["Close"], length=10)
df["ema_20"] = ta.ema(df["Close"], length=20)
df["ema_50"] = ta.ema(df["Close"], length=50)
df["ema_100"] = ta.ema(df["Close"], length=100)
df["ema_200"] = ta.ema(df["Close"], length=50)

# Stochastic Oscillator
stoch = ta.stoch(df["High"], df["Low"], df["Close"])
df["stoch_k"] = stoch["STOCHk_14_3_3"]
df["stoch_d"] = stoch["STOCHd_14_3_3"]

df["roc"] = ta.roc(close=df["Close"], length=10)

adx = ta.adx(
    high=df["High"],
    low=df["Low"],
    close=df["Close"],
    length=14
)
df["adx"]  = adx["ADX_14"]
df["di_plus"]  = adx["DMP_14"] 
df["di_minus"] = adx["DMN_14"]  

df["atr_14"] = ta.atr(
    high=df["High"],
    low=df["Low"],
    close=df["Close"],
    length=14
)
df["atr_20"] = ta.atr(
    high=df["High"],
    low=df["Low"],
    close=df["Close"],
    length=20
)

df["close_pos"] = (df["Close"] - df["Low"]) / (df["High"] - df["Low"])

df["body_range_ratio"] = (df["Close"] - df["Open"]).abs() / (df["High"] - df["Low"])

# --- Volume Z-Score(50) ---
# df["volume_zscore_50"] = (df["Volume"] - df["Volume"].rolling(50).mean()) / df["Volume"].rolling(50).std()

# --- Bollinger Bands(20) ---
bb = ta.bbands(df["Close"], length=20, std=2)

col_lower  = next(c for c in bb.columns if c.startswith("BBL_20"))
col_middle = next(c for c in bb.columns if c.startswith("BBM_20"))
col_upper  = next(c for c in bb.columns if c.startswith("BBU_20"))

df["bb_lower_20"] = bb[col_lower]
df["bb_middle_20"] = bb[col_middle]
df["bb_upper_20"]  = bb[col_upper]

# --- Bollinger Bands Width (20) ---
df["bb_width_20"] = (df["bb_upper_20"] - df["bb_lower_20"]) / df["bb_middle_20"]

# --- On-Balance Volume (OBV) ---
# df["obv"] = ta.obv(df["Close"], df["Volume"])

df["returns"] = df["Close"].pct_change()
df['direction'] = np.where(df['returns'] > 0, 1, 0)

df.dropna(inplace=True)

print(tb.tabulate(df.head(), headers='keys', tablefmt='psql'))
print(tb.tabulate(df.tail(), headers='keys', tablefmt='psql'))

df.to_csv(PATH_OUT)

+---------------------+---------+---------+---------+---------+------------+----------+----------+----------+---------+----------+----------+----------+----------+-----------+-----------+-----------+-----------+----------+---------+-----------+------------+----------+----------+-------------+--------------------+---------------+----------------+---------------+---------------+--------------+-------------+
| Datetime            |    Open |    High |     Low |   Close |   % change |   rsi_14 |   rsi_28 |   rsi_50 |   rsi_7 |     macd |   ema_10 |   ema_20 |   ema_50 |   ema_100 |   ema_200 |   stoch_k |   stoch_d |      roc |     adx |   di_plus |   di_minus |   atr_14 |   atr_20 |   close_pos |   body_range_ratio |   bb_lower_20 |   bb_middle_20 |   bb_upper_20 |   bb_width_20 |      returns |   direction |
|---------------------+---------+---------+---------+---------+------------+----------+----------+----------+---------+----------+----------+----------+----------+-----------+-------